<a href="https://colab.research.google.com/github/vilcagamarracf/Inv_arroz_METRIC/blob/main/Inv_Arroz_METRIC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center>
<br>
<img src="https://github.com/vilcagamarracf/Inv_arroz_METRIC/blob/main/Github%20Cover%20-%20METRIC.png?raw=true"/>
<h1>Aplicación del modelo METRIC en cultivos de Arroz, Lambayeque</h1>
<h2> Investigación 2021 </h2>
</center>

**Information about this project**
```
FILE          : Inv-Arroz-METRIC.ipynb
AUTHOR        : Cesar Francisco Vilca Gamarra 
EMAIL         : vilcagamarracf@gmail.com 
ORGANIZATION  : UNALM 
CREATION DATE : April 29, 2021 
DEPENDENCIES  : earthengine-api, os, folium, pandas, matplotlib, numpy ...  
PURPOSE       : Download Landsat Products to estimate spatial ET
```


# Descripción

En esta libreta se realizará la exploración, selección y procesamiento de imágenes satelitales del producto Landsat 8 SR usando Google Earth Engine en Google Colab (Python) con el objetivo de replicar el modelo METRIC en cultivos de arroz para un área de estudio en el departamento de Lambayeque.

**Problemática**

- Estimación de ET común:
  $$
  \text{ET} = \text{ET}_{ref} * K_c
  $$
  Dónde:
  - $\text{ET}$ : Evapotranspiración
  - $\text{ET}_{ref}$ : Evapotranspiración de referencia basada en el clima
  - $K_c$: Coeficiente de cultivo (de acuerdo al tipo y etapa de crecimiento)

- Dificultades de la estimación común:
  - Confiabilidad de usar valores idealizados de $K_c$ ya que se generaron para ciertas condiciones reales de crecimiento y vegetación.
  Identificación de etapas vegetativas y condiciones de crecimiento para comparación con valores de $K_c$
  - Predicción correcta de fechas según etapa de crecimiento para cultivos de grandes extensiones (ya que 

- Una solución:
  - **Los satélites son capaces de obtener información espacial de evapotranspiración de numerosas extensiones a partir de técnicas de balance de energía.**  

  > Estimación de la evapotranspiración de los cultivos bajo riego mediante imágenes de sensores remotos (tipo de sensor y bandas del espectro electromagnético)


**¿Por qué medir/estimar la evapotranspiración?**

- La evapotranspiración (ET) es un componente esencial en modelos hidrológicos y de circulación general.
- Es usado para inferir la humedad del suelo, como dato de entrada para pronósticos climáticos y de inundaciones.



**Métodos de estimación**

En Operational Remote Sensing of ET and Challenges [(2011)](https://www.intechopen.com/books/evapotranspiration-remote-sensing-and-modeling/operational-remote-sensing-of-et-and-challenges) se menciona a Kalma et al. (2008), donde reunen las **metodologías existentes** para estimar la evapotranspiración de los cultivos usando Teledetección, tales como:
  - **Balance Energético de la Superficie**
  - Métodos estadísticos que utilizan diferencias entre la temperatura de la superficie y el aire
  - Correlaciones simplificadas o relaciones entre extremos de temperatura superficial de una imagen y puntos finales del ET anticipado
  - ET relativa basada en la vegetación que se multiplica por una ET de referencia basada en el tiempo

En la presente investigación, profundizaremos el método de Balance Energético de la Superficie.

**Métodos: Balance Energético de la Superficie**

Se subdivide en:
-  Balance energético completo para la imagen satelital: 
    \begin{equation}\lambda{E}=R_n - G -H \end{equation}
     dónde:
     - $\lambda{E}$ es la densidad latente del flujo térmico, representando la energía "consumida" por la evaporación del agua, 
     - $R_n$ es la densidad neta del flujo de radiación, 
     - $G$ es la densidad del flujo de calor del suelo, y 
     - $H$ es la densidad sensible del flujo de calor al aire.
- Índice de estrés hídrico basado en la temperatura superficial y las cantidades de vegetación. 
- Aplicación de un Modelo Continuo de Superficie Terrestre (MSL) que se inicializa parcialmente y avanzado, en el tiempo, utilizando imágenes satelitales

Todas las metodologías anteriormente mencionadas solamente trabajan sobre imágenes disponibles y dependiendo de la revisita del mismo, quedan vacíos de información entre imágenes.

**Modelo METRIC** 

> METRIC uses TIR-Multispectral satellite images (e.g., from Landsat 7 ETM+ and 8 OLI) and ground-based meteorological data to estimate pixel-based daily $ET_a$ at 30 × 30 m spatial resolution via a surface energy balance equation at the time of the satellite overpass 

<center>
<img src= 'https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Fimage.slideserve.com%2F327001%2Fhow-metric-works6-l.jpg&f=1&nofb=1' width=50%/>
</center>

Datos principales:
- Imágenes termales satelitales de onda corta y larga (Landsat 7 ETM+ y Landsat 8 OLI/TIRS)
- Un modelo digital de elevación (DEM)
- Datos de medición de campo de la zona de interés o cercana.

A continuación procederemos con la parte práctica usando la API de Python de Earth Engine con Colab. 

# METRIC con Earth Engine en Colab

## 1. Inicio

- Credenciales 
- Librerías y Funciones 
- ROI Lambayeque

Primero ingresamos nuestros datos de Google Earth Engine (necesario una cuenta registrada).

In [1]:
# Ingresa tus credenciales de Google Earth Engine para comenzar
import ee
ee.Authenticate()

# Inicio
ee.Initialize()

# Mensaje de login
print('\nYa estás logeado.')
print('GEE Version:', ee.__version__)

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=G1dv1MrUnjNt4Ipe4r4WPDdr_JQbvt1JiwZvJmUhZeM&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWjVpYn3ytnbHo_HReQDQhzM4xkYbnQBIplg_75AUr7c1AzNEVbnjQo

Successfully saved authorization token.

Ya estás logeado.
GEE Version: 0.1.283


Luego importamos las librerías y funciones necesarias para desarrollar la parte práctica.

In [2]:
# @markdown Importar librerías, funciones y más:
# @markdown - `mapdisplay`: Variable global: `mapViz`
# @markdown - `ee_get_dates`
# @markdown - `conv_fecha`
# @markdown - `ver_rangos`
# @markdown - `create_dateranges`

# mapdisplay ----------------------------------------
import folium 
def mapdisplay(center, dicc, Tiles="OpensTreetMap",zoom_start=10):
    '''
    Crea mapas interactivos usando la librería `folium`

    - `center`: Center of the map (Latitude and Longitude) (`list`)
    - `dicc`: Earth Engine Geometries or Tiles dictionary (`dict`)
    - `Tiles=`: 
    Mapbox Bright 
    Mapbox Control Room
    Stamen Terrain
    Stamen Toner
    stamenwatercolor
    cartodbpositron
    - `zoom_start=`: Initial zoom level for the map (`int`)

    Return: A folium.Map object
    '''
    # Centrado de imagen (El [::-1] lee inversamente la lista)
    center = center[::-1]

    # Agregando parámetros de visualización
    global mapViz
    mapViz = folium.Map(
        location=center,
        tiles=Tiles, zoom_start=zoom_start, control_scale=True)

    # Seteando la visualización de Images
    for k,v in dicc.items():
      if ee.image.Image in [type(x) for x in v.values()]:
        folium.TileLayer(tiles = v["tile_fetcher"].url_format,
                         attr  = 'Google Earth Engine',
                         overlay =True,
                         name  = k).add_to(mapViz)
      else:
        folium.GeoJson(data = v,
                       name = k).add_to(mapViz)
    # Agregar: Minimapa
    # from folium.plugins import MiniMap
    # minimap = MiniMap(
    #     tile_layer="OpensTreetMap", 
    #     position="bottomright", 
    #     zoom_level_fixed=4)
    # mapViz.add_child(minimap) # https://nbviewer.jupyter.org/github/python-visualization/folium/blob/master/examples/MiniMap.ipynb

    # Fullscreen (no funciona en colab)
    # plugins.Fullscreen().add_to(mapViz)

    # Agregando el control de layers
    mapViz.add_child(folium.LayerControl())
    return mapViz
print('mapdisplay(center, dicc, Tiles="OpensTreetMap",zoom_start=10) listo!')


# ee_get_dates ----------------------------------------
from datetime import datetime as dt
def ee_get_dates(ic):
  """
  Permite obtener información de la Fecha y Hora de toma de imágen por el satélite.
  - ic: `ee.ImageCollection()`
  """
  dates = ic.aggregate_array('system:time_start').getInfo()
  anonym = lambda x: dt.utcfromtimestamp(x/1000).strftime('%Y-%m-%d %H:%M:%S')
  return list(map(anonym, dates))
print('ee_get_dates(ic) listo!')

 
# conv_fecha ----------------------------------------
def conv_fecha(img, exact=None):
  '''
  Obtener fecha con formato YYYY-MM-dd H:mm:ss
  img: ee.Image
  '''
  if exact==True:
    return img.date().format('YYYY-MM-dd').getInfo()
  else:
    return img.date().format('YYYY-MM-dd H:mm:ss').getInfo()
print('conv_fecha(img, getInfo=None) listo!')


# ver_rangos ----------------------------------------

Start_date = '2012-01-01'
End_date   = '2021-01-31'
range_date = [Start_date, End_date]

def ver_rangos(img_col, range_date= [Start_date, End_date]):
  """
  Devuelve los rangos de fechas existente en un rango inicial escrito manualmente como input.
  Pasos:
  1. `Reducer.minMax()` devuelve un valor min y max
  2. `icol.reduceColumns()` devuelve un diccionario con el min-max
  more info: https://developers.google.com/earth-engine/apidocs/ee-imagecollection-reducecolumns?hl=en
  """
  
  rango = img_col.reduceColumns(ee.Reducer.minMax(), ["system:time_start"]) # Retorna un ee.dictionary: {'max': 1608132402761, 'min': 1545060401000}

  # Obtención de la fecha min y max del image collection en formato ISO standard 8601
  # Javascript trabaja las fechas con milisegundos (se deja así)
  # Python     trabaja las fechas con segundos (por eso /1000)
  init_date = ee.Date(rango.get('min')).getInfo()['value']/1000.
  last_date = ee.Date(rango.get('max')).getInfo()['value']/1000.

  # Dar formato a las fechas  
  from datetime import datetime as dt
  init_date_f = dt.utcfromtimestamp(init_date).strftime('%Y-%m-%d') # %H:%M:%S
  last_date_f = dt.utcfromtimestamp(last_date).strftime('%Y-%m-%d') # %H:%M:%S
  
  # Obtenga el rango de fechas de las imágenes en la colección.
  range_date = [Start_date, End_date]
  print('Total imágenes: {}'.format(img_col.size().getInfo()))
  print('Rango Temporal Ingresado : {} - {}'.format(range_date[0], range_date[1]))
  print('Rango Temporal Real      : {} - {}'.format(init_date_f, last_date_f))
print('ver_rangos(icol, range_date= [Start_date, End_date]) listo!')


# create_dateranges ----------------------------------------
def create_dateranges(init_date, time, range_1, range_2):
  '''
  init_date: '20XX' for monthly dates, '20XX-XX' for daily dates(string)
  time: 'year', 'month', 'day' (string)
  range_1: 1, ... 12 for monthly values (int)
  range_2: 1, ... 13 for monthly values (int)

  Returns a date ranges in a list form.
  '''
  ranges = []
  def create_dates(date, plus_time):
     ''' 
     init_date: string values only, example:  '2013-1'
     time: 'year', 'month', 'day'
  
     more info: https://developers.google.com/earth-engine/apidocs/ee-date-getrange
     '''
     var_time = ee.Date(date).getRange(plus_time).getInfo().get('dates')
  
     from datetime import datetime as dt #-%d
     Start_Date = dt.utcfromtimestamp(var_time[0]/1000).strftime('%Y-%m')
     End_Date   = dt.utcfromtimestamp(var_time[1]/1000).strftime('%Y-%m')
     return  [Start_Date, End_Date, i-1]
  
  for i in range(range_1, range_2+1):
    ranges.append(create_dates(init_date+'-'+str(i), time))
  return ranges
print('create_dateranges(init_date, time, range_1, range_2) listo!')

mapdisplay(center, dicc, Tiles="OpensTreetMap",zoom_start=10) listo!
ee_get_dates(ic) listo!
conv_fecha(img, getInfo=None) listo!
ver_rangos(icol, range_date= [Start_date, End_date]) listo!
create_dateranges(init_date, time, range_1, range_2) listo!


Una vez ingresado nuestros datos de Google Earth Engine ya podemos visualizar la zona de estudio de forma interactiva con [`folium`](https://python-visualization.github.io/folium/) en Python.

In [3]:
# @markdown *Visualización: Departamento de Lambayeque*

# 24 Departamentos del Perú
shpPeru = ee.FeatureCollection('users/CesarVilca/Departamentos_Peru')

# Departamento de Lambayeque
shpDep = shpPeru.filter(ee.Filter.eq('NOMBDEP', 'LAMBAYEQUE'))

# Geometrías
roiPeru = shpPeru.geometry()
roiDep  = shpDep.geometry()

# Centroides
centroidPeru = roiPeru.centroid().coordinates().getInfo()   
centroidDep = roiDep.centroid().coordinates().getInfo()

# Visualización de FeatureCollection con .draw
# https://htmlcolors.com/ para modificar el color
roiPeruDraw = shpPeru.draw(color = "000000", strokeWidth = 2, pointRadius = 2) 
roiDepDraw  = shpDep.draw(color = "00FF00", strokeWidth = 2, pointRadius = 2) 

dicc_viz1 = {
    'Perú': roiPeruDraw.getMapId(),
    'Lambayeque': roiDepDraw.getMapId(),
    }  
    
mapdisplay(centroidDep, dicc_viz1, zoom_start=9)

# print('Zona de estudio lista!')

## 2. Exploración de dataset "Landsat 8 Surface Reflectance"


### 2.1. Constelación Landsat

**Landsat** 

Programa conjunto del USGS y la NASA que ha estado observando la Tierra continuamente desde 1972 hasta la actualidad. Hoy en día, los satélites Landsat obtienen imágenes de toda la superficie de la Tierra a una resolución de 30 metros aproximadamente una vez cada dos semanas, incluidos datos multiespectrales y térmicos. El USGS produce datos en 3 categorías para cada satélite (Nivel 1, Nivel 2 y RT).


**Landsat Collections**

Landsat se divide en dos colecciones:

- ***Landsat Collection 1***: Inició en 2016 y contiene toda la data de Level-1 adquirida desde 1972 hasta el presente de Landsat 1-8.
  - En Earth Engine: [USGS Landsat 8 Surface Reflectance Tier 1](https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C01_T1_SR)
- ***Landsat Collection 2***: Inició a mediados del año 2021 y representa un hito en la historia de Landsat ya que realiza mejoras sustanciales al procesamiento de las imágenes antes adquiridas, dando como resultado un mayor aprovechamiento de los avances recientes en el procesamiento de datos, el desarrollo de algoritmos y las capacidades de acceso y distribución de datos.
  - En Earth Engine: [USGS Landsat 8 Level 2, Collection 2, Tier 1](https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C02_T1_L2)

**Nota**: A partir del primero de enero del 2022, todas las imágenes adquiridas y procesadas serán almacenadas solamente en el inventario *Landsat Collection 2*. [Fuente: USGS](https://www.usgs.gov/core-science-systems/nli/landsat/landsat-collections?qt-science_support_page_related_con=2#qt-science_support_page_related_con)


**Landsat 8 en Earth Engine**

A continuación veremos las propiedades de las bandas del producto Landsat 8 Surface Reflectance Tier 1 en el dataset de Earth Engine. 

\

|Name	|Units|	Scale|	Wavelength|	Description|
|:-:|:-:|-|-|-|
|B1		|-|0.0001	|0.435-0.451 μm	|Band 1 (ultra blue) surface reflectance|
|B2		|-|0.0001	|0.452-0.512 μm	|Band 2 (blue) surface reflectance|
|B3		|-|0.0001	|0.533-0.590 μm	|Band 3 (green) surface reflectance|
|B4		|-|0.0001	|0.636-0.673 μm	|Band 4 (red) surface reflectance|
|B5		|-|0.0001	|0.851-0.879 μm	|Band 5 (near infrared) surface reflectance|
|B6		|-|0.0001	|1.566-1.651 μm	|Band 6 (shortwave infrared 1) surface reflectance|
|B7		|-|0.0001	|2.107-2.294 μm	|Band 7 (shortwave infrared 2) surface reflectance|
|B10	|Kelvin	|0.1	|10.60-11.19 μm|Band 10 brightness temperature.|
|B11	|Kelvin	|0.1	|11.50-12.51 μm|Band 11 brightness temperature.|

\

**Nota**:
Las bandas B10 y B11, aunque se recopilaron originalmente con una resolución de 100 m / píxel, se han vuelto a muestrear mediante convolución cúbica a 30 m.

### 2.2. Exploración rápida del dataset Landsat 8 Surface Reflectance

Se realizará un filtrado al dataset con el fin de reducir la cantidad de imágenes a las que abarquen nuestra área de estudio. Es posible el filtrado mediante: 
- Un punto: coordenadas como longitud y latitud de un lugar exacto
- Multipuntos: archivo shapefile como conjunto de coordenadas
- Propiedades: Path y Row
  - **Nota**: Para ubicar el Path y Row de la zona de estudio de forma interactiva, usar el siguiente enlace: [GEO GPS PERÚ: Ubica el PATH y ROW de tu distrito para imágenes LANDSAT](https://www.geogpsperu.com/2016/02/ubica-el-path-y-row-de-tu-distrito-para.html)


In [4]:
# @markdown **Prueba**: Visualización rápida de imágenes Landsat para el ROI

# @markdown Seleccionamos las fechas de inicio y fin para el filtrado

# Inputs
Start_date = '2020-01-01'  #@param {type: "date"}
End_date   = '2021-01-01'  #@param {type: "date"}
range_date = [Start_date, End_date]

## Filtrado de imágenes
# 1er Filtrado: Mejor fecha de toma
img_col = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR")\
            .filterDate(Start_date, End_date)\
            .filterBounds(roiDep)\
            .sort('CLOUD_COVER')

mejor_fecha = conv_fecha(img_col.first(), exact=True)
fecha1 = ee.Date(mejor_fecha)
fecha2 = fecha1.advance(16, 'day') # ee.Date().advance()

# Metadatos
print('--- Metadatos del Image Collection img_col ---')
ver_rangos(img_col)
print('Fecha de mejor toma: ', mejor_fecha)

# 2do Fitrado: Visualización de imágenes con mejor toma
img_col = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR")\
            .filterDate(fecha1, fecha2)\
            .filterBounds(roiDep)\
            .sort('CLOUD_COVER')

imagenes = img_col.median().multiply(0.0001).select(['B4','B3','B2'])

# Visualizacion
chongoyape = ee.Geometry.Point([-79.389, -6.639])
oyotun     = ee.Geometry.Point([-79.309, -6.846])
centroide  = ee.Geometry.Point([-79.823, -6.351])

dict_img = {
    'img_col': imagenes.getMapId({
        'min':0.0, 'max':0.3, 'bands': ['B4','B3','B2']}),
    'Lambayeque_shp': roiDepDraw.getMapId(),
    'Chongoyape': chongoyape.getInfo(),
    'Centroide': centroide.getInfo(),
    'Oyotun': oyotun.getInfo()
    } # ,'roi': roi_clip.getInfo()

mapdisplay(centroidDep, dict_img, zoom_start=8)

--- Metadatos del Image Collection img_col ---
Total imágenes: 77
Rango Temporal Ingresado : 2020-01-01 - 2021-01-01
Rango Temporal Real      : 2020-01-07 - 2020-12-26
Fecha de mejor toma:  2020-04-21


Veremos que la zona abarca 4 imágenes Landsat 8.

In [5]:
# Ver fechas de las imágenes anteriormente mostradas
ee_get_dates(img_col)

['2020-04-21 15:27:50',
 '2020-04-21 15:27:26',
 '2020-04-28 15:33:34',
 '2020-04-30 15:21:35']

A continuación veremos las dos imágenes que ofrecen mayor información del departamento de Lambayeque.

In [6]:
# @markdown Visualización de dos imágenes

# Inputs
Start_date = '2020-04-21'  
End_date   = '2020-04-22'  
range_date = [Start_date, End_date]

# Filtrado de imágenes
img_col1 = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR")\
        .filterDate(Start_date, End_date)\
        .filterBounds(ee.Geometry.Point(-79.823, -6.351))\
        .sort('CLOUD_COVER')#\.first()

img_col2 = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR")\
        .filterDate(Start_date, End_date)\
        .filterBounds(ee.Geometry.Point(-79.309, -6.846))\
        .sort('CLOUD_COVER')#\.first()

bands = ['B4','B3','B2']
img_rgb1 = img_col1.first().multiply(0.0001).select(bands)   #.clip(roi_clip)
img_rgb2 = img_col2.first().multiply(0.0001).select(bands)   #.clip(roi_clip)

## Metadatos 

# Imagen superior
print('Rango de fechas disponibles para Landsat8')
ver_rangos(img_col1, range_date=range_date)

print('\n--- Imagen Superior ---')
print('Mejor imagen:')
print('% nubosidad:', img_col1.first().getInfo()['properties']['CLOUD_COVER'])
print('Image ID:',img_col1.first().getInfo().get('id'))
print('Fecha:', conv_fecha(img_col1.first()))

# Imagen Inferior
print('\n--- Imagen Inferior ---')
print('Mejor imagen:')
print('% nubosidad:', img_col2.first().getInfo()['properties']['CLOUD_COVER'])
print('Image ID:',img_col2.first().getInfo().get('id'))
print('Fecha:', conv_fecha(img_col2.first()))

## Visualización
chongoyape = ee.Geometry.Point([-79.389, -6.639])
oyotun     = ee.Geometry.Point([-79.309, -6.846])
centroide  = ee.Geometry.Point([-79.823, -6.351])

dict_getMapId = {'min':0.0, 'max':0.3, 'bands': bands}

dict_img = {
    'imgSuperior': img_rgb1.clip(roiDep).getMapId(dict_getMapId),
    'imgInferior': img_rgb2.clip(roiDep).getMapId(dict_getMapId),
    # 'Lambayeque': roiDepDraw.getMapId(),
    'Chongoyape': chongoyape.getInfo(),
    #'Centroide': centroide.getInfo(),
    'Oyotun': oyotun.getInfo()
    }

mapdisplay(centroidDep, dict_img, zoom_start=9)

Rango de fechas disponibles para Landsat8
Total imágenes: 1
Rango Temporal Ingresado : 2020-04-21 - 2020-04-22
Rango Temporal Real      : 2020-04-21 - 2020-04-21

--- Imagen Superior ---
Mejor imagen:
% nubosidad: 6.67
Image ID: LANDSAT/LC08/C01/T1_SR/LC08_010064_20200421
Fecha: 2020-04-21 15:27:26

--- Imagen Inferior ---
Mejor imagen:
% nubosidad: 1.66
Image ID: LANDSAT/LC08/C01/T1_SR/LC08_010065_20200421
Fecha: 2020-04-21 15:27:50


## 3. Selección de imágenes: Landsat 8

- Visualización de imágenes satelitales para el año 2020
- Elaboración de una tabla con la información de las imágenes respectivas

### 3.2. Visualización de imágenes a partir de sus IDs

#### Generar IDs de imágenes Landsat de forma mensual (2020)

Mediante los diccionarios (estructura de datos en Python) es posible organizar información en su interior, de esta forma podremos realizar una posterior visualización de manera ordenada. 

A continuación asignaremos los IDs de cada imágen a el número del mes en el que fué realizada la captura de la misma.



In [ ]:
# @markdown Generador de IDs Anual: **`imgCol_dict`**

# @markdown **No es necesario correr esta celda de código** 
# @markdown pues se ha provisto ya la lista de IDs en la siguiente sección.

# Inputs
long = -79.809 #@param [] {type:"raw", allow-input: true}
lat = -6.746 #@param [] {type:"raw", allow-input: true}
coordenada = list([long, lat])
year =  2020 #@param {type:"integer"}
ID_snippet_name = "LANDSAT/LC08/C01/T1_SR"

# Filtrado
periodo = create_dateranges(str(year),'month', 1, 12)

imgCol = [ee.ImageCollection(ID_snippet_name)\
            .filterDate(periodo[i][0], periodo[i][1])\
            .filterBounds(ee.Geometry.Point(coordenada)) for i in range(12)] 

# Cantidad de imágenes por mes 
imgCol_size = [imgCol[i].size().getInfo() for i in range(12)]

# Diccionario de Meses con su imágenes correspondientes
imgCol_dict = {}

for i in range(12):
  IDs = []
  imgCol_dict[i+1] = IDs
  for j in range(imgCol_size[i]):
    IDs.append(imgCol[i].getInfo()['features'][j]['id'])

imgCol_dict

{1: ['LANDSAT/LC08/C01/T1_SR/LC08_010065_20200116'],
 2: ['LANDSAT/LC08/C01/T1_SR/LC08_010065_20200201',
  'LANDSAT/LC08/C01/T1_SR/LC08_010065_20200217'],
 3: ['LANDSAT/LC08/C01/T1_SR/LC08_010065_20200304',
  'LANDSAT/LC08/C01/T1_SR/LC08_010065_20200320'],
 4: ['LANDSAT/LC08/C01/T1_SR/LC08_010065_20200405',
  'LANDSAT/LC08/C01/T1_SR/LC08_010065_20200421'],
 5: ['LANDSAT/LC08/C01/T1_SR/LC08_010065_20200507',
  'LANDSAT/LC08/C01/T1_SR/LC08_010065_20200523'],
 6: ['LANDSAT/LC08/C01/T1_SR/LC08_010065_20200608',
  'LANDSAT/LC08/C01/T1_SR/LC08_010065_20200624'],
 7: ['LANDSAT/LC08/C01/T1_SR/LC08_010065_20200726'],
 8: ['LANDSAT/LC08/C01/T1_SR/LC08_010065_20200811',
  'LANDSAT/LC08/C01/T1_SR/LC08_010065_20200827'],
 9: ['LANDSAT/LC08/C01/T1_SR/LC08_010065_20200912',
  'LANDSAT/LC08/C01/T1_SR/LC08_010065_20200928'],
 10: ['LANDSAT/LC08/C01/T1_SR/LC08_010065_20201014',
  'LANDSAT/LC08/C01/T1_SR/LC08_010065_20201030'],
 11: ['LANDSAT/LC08/C01/T1_SR/LC08_010065_20201115'],
 12: ['LANDSAT/LC08/C01

#### Visualización de imágenes Landsat (2020)

Con el diccionario ya listo es posible proceder a la visualización. Con ayuda de `ipywidgets` es posible realizar visualizaciones interactivas con HTML siendo el indicado para ello el módulo `interact`. 

Más información:
- [ipywidgets](https://ipywidgets.readthedocs.io/en/latest/examples/Using%20Interact.html#)
- [interact en Colab](https://colab.research.google.com/github/jupyter-widgets/ipywidgets/blob/master/docs/source/examples/Using%20Interact.ipynb)

##### Landsat 8 T1_L2 

- [Dataset source](https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C02_T1_L2)

In [ ]:
# @markdown Diccionario de imágenes Landsat 8 T1_L2: `imgCol_dict`

imgCol_dict = {
    1: ['LANDSAT/LC08/C02/T1_L2/LC08_010065_20200116'],
    2: ['LANDSAT/LC08/C02/T1_L2/LC08_010065_20200201',
      'LANDSAT/LC08/C02/T1_L2/LC08_010065_20200217'],
    3: ['LANDSAT/LC08/C02/T1_L2/LC08_010065_20200304',
      'LANDSAT/LC08/C02/T1_L2/LC08_010065_20200320'],
    4: ['LANDSAT/LC08/C02/T1_L2/LC08_010065_20200405',
      'LANDSAT/LC08/C02/T1_L2/LC08_010065_20200421'],
    5: ['LANDSAT/LC08/C02/T1_L2/LC08_010065_20200507',
      'LANDSAT/LC08/C02/T1_L2/LC08_010065_20200523'],
    6: ['LANDSAT/LC08/C02/T1_L2/LC08_010065_20200608',
      'LANDSAT/LC08/C02/T1_L2/LC08_010065_20200624'],
    7: ['LANDSAT/LC08/C02/T1_L2/LC08_010065_20200726'],
    8: ['LANDSAT/LC08/C02/T1_L2/LC08_010065_20200811',
      'LANDSAT/LC08/C02/T1_L2/LC08_010065_20200827'],
    9: ['LANDSAT/LC08/C02/T1_L2/LC08_010065_20200912',
      'LANDSAT/LC08/C02/T1_L2/LC08_010065_20200928'],
    10: ['LANDSAT/LC08/C02/T1_L2/LC08_010065_20201014',
      'LANDSAT/LC08/C02/T1_L2/LC08_010065_20201030'],
    11: ['LANDSAT/LC08/C02/T1_L2/LC08_010065_20201115'],
    12: ['LANDSAT/LC08/C02/T1_L2/LC08_010065_20201201',
      'LANDSAT/LC08/C02/T1_L2/LC08_010065_20201217']
      }

In [ ]:
# @markdown Visualización de imágenes con `interact`

from ipywidgets import interact 
from pprint import pprint

Natural_Color  = ['SR_B4', 'SR_B3', 'SR_B2']

def visualizar_mes(mes):

  # Diccionario para visualización
  viz_dict = {}
  scale = 0.0000275
  # Generar diccionario de visualización
  for i in range(len(imgCol_dict[mes])):
    viz_dict[conv_fecha(ee.Image(imgCol_dict[mes][i]))] \
      = ee.Image(imgCol_dict[mes][i])\
      .multiply(scale).add(-0.2)\
      .clip(roiDep)\
      .getMapId({'min':0,'max':0.3, 'bands':Natural_Color})

  # Presentar ID's
  print("\nID's disponibles:")
  pprint(imgCol_dict[mes])
  print('')

  # Mostrar el mapa leaflet
  return mapdisplay([-79.809, -6.746], viz_dict, zoom_start=10)
  
mes = [i for i in range(1,12+1)]

interact(visualizar_mes, mes=mes);

interactive(children=(Dropdown(description='mes', options=(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12), value=1), O…

##### Landsat 8 SR

- [Dataset source](https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C01_T1_SR)

In [ ]:
# @markdown Diccionario de imágenes Landsat 8 SR: `imgCol_dict_SR`

imgCol_dict_SR = {
    1: ['LANDSAT/LC08/C01/T1_SR/LC08_010065_20200116'],
    2: [
        'LANDSAT/LC08/C01/T1_SR/LC08_010065_20200201',
        'LANDSAT/LC08/C01/T1_SR/LC08_010065_20200217'
    ],  # buena foto
    3: [
        'LANDSAT/LC08/C01/T1_SR/LC08_010065_20200304',
        'LANDSAT/LC08/C01/T1_SR/LC08_010065_20200320'
    ],
    4: [
        'LANDSAT/LC08/C01/T1_SR/LC08_010065_20200405',
        'LANDSAT/LC08/C01/T1_SR/LC08_010065_20200421'
    ],  # buena foto
    5: [
        'LANDSAT/LC08/C01/T1_SR/LC08_010065_20200507',
        'LANDSAT/LC08/C01/T1_SR/LC08_010065_20200523'
    ],
    6: [
        'LANDSAT/LC08/C01/T1_SR/LC08_010065_20200608',
        'LANDSAT/LC08/C01/T1_SR/LC08_010065_20200624'
    ],
    7: ['LANDSAT/LC08/C01/T1_SR/LC08_010065_20200726'],
    8: [
        'LANDSAT/LC08/C01/T1_SR/LC08_010065_20200811',
        'LANDSAT/LC08/C01/T1_SR/LC08_010065_20200827'
    ],
    9: [
        'LANDSAT/LC08/C01/T1_SR/LC08_010065_20200912',
        'LANDSAT/LC08/C01/T1_SR/LC08_010065_20200928'
    ],
    10: [
        'LANDSAT/LC08/C01/T1_SR/LC08_010065_20201014',
        'LANDSAT/LC08/C01/T1_SR/LC08_010065_20201030'
    ],
    11: ['LANDSAT/LC08/C01/T1_SR/LC08_010065_20201115'],
    12: ['LANDSAT/LC08/C01/T1_SR/LC08_010065_20201201']
}


In [ ]:
# @markdown Visualización de imágenes con `interact`

from ipywidgets import interact 
from pprint import pprint

Natural_Color  = ['B4', 'B3', 'B2']

def visualizar_mes(mes):

  # Diccionario para visualización
  viz_dict = {}

  # Generar diccionario de visualización
  for i in range(len(imgCol_dict_SR[mes])):
    viz_dict[conv_fecha(ee.Image(imgCol_dict_SR[mes][i]))] \
      = ee.Image(imgCol_dict_SR[mes][i])\
      .multiply(0.0001)\
      .clip(roiDep)\
      .getMapId({'min':0,'max':0.3, 'bands':Natural_Color})

  # Presentar ID's
  print("\nID's disponibles:")
  pprint(imgCol_dict_SR[mes])
  print('')

  # Mostrar el mapa leaflet
  return mapdisplay([-79.809, -6.746], viz_dict, zoom_start=10)
  
mes = [i for i in range(1,12+1)]

interact(visualizar_mes, mes=mes);

interactive(children=(Dropdown(description='mes', options=(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12), value=1), O…

### 3.3. Generar tabla de propiedades

Se va a generar una tabla con información sobre las imágenes seleccionadas anteriormente mediante la librería `pandas`.

In [ ]:
# @markdown Generar tabla de resultados con `pandas`

long = -79.809 #@param [] {type:"raw", allow-input: true}
lat = -6.746 #@param [] {type:"raw", allow-input: true}
coordenada = list([long, lat])

year =  2020 #@param {type:"integer"}
Start_date = str(year) + '-01-01' 
End_date   = str(year+1) + '-01-01' 
range_date = [Start_date, End_date]

periodo = create_dateranges(str(year),'month', 1, 12)

ID_snippet_name = "LANDSAT/LC08/C01/T1_SR" 

icol_sr = ee.ImageCollection(ID_snippet_name)\
            .filterDate(range_date[0], range_date[1])\
            .filterBounds(ee.Geometry.Point(coordenada))

lista_fechas = icol_sr.aggregate_array('system:time_start').getInfo()
cantidad = len(lista_fechas)

# Generar tabla con pandas
import pandas as pd
df = pd.DataFrame(lista_fechas, columns = ['millis'])

def add_date_info(df):
  df['Número'] = [i for i in range(1, cantidad + 1)]
  df['Timestamp'] = pd.to_datetime(df['millis'], unit='ms')
  df['Día Juliano'] = pd.DatetimeIndex(df['Timestamp']).dayofyear
  df['Sensor'] = ['OLI/TIRS' for i in range(1, cantidad + 1)]
  df['Año'] = pd.DatetimeIndex(df['Timestamp']).year
  df['Mes'] = pd.DatetimeIndex(df['Timestamp']).month
  df['Día'] = pd.DatetimeIndex(df['Timestamp']).day
  
  df = df.drop(columns=['millis'])
  df = df.drop(columns=['Timestamp'])
  return df

df = add_date_info(df)
print(df.to_string(index=False))

# Exportar como .csv 
# df.to_csv('datos_2020_L8.csv')
# lista_fechas

 Número  Día Juliano    Sensor   Año  Mes  Día
      1           16  OLI/TIRS  2020    1   16
      2           32  OLI/TIRS  2020    2    1
      3           48  OLI/TIRS  2020    2   17
      4           64  OLI/TIRS  2020    3    4
      5           80  OLI/TIRS  2020    3   20
      6           96  OLI/TIRS  2020    4    5
      7          112  OLI/TIRS  2020    4   21
      8          128  OLI/TIRS  2020    5    7
      9          144  OLI/TIRS  2020    5   23
     10          160  OLI/TIRS  2020    6    8
     11          176  OLI/TIRS  2020    6   24
     12          208  OLI/TIRS  2020    7   26
     13          224  OLI/TIRS  2020    8   11
     14          240  OLI/TIRS  2020    8   27
     15          256  OLI/TIRS  2020    9   12
     16          272  OLI/TIRS  2020    9   28
     17          288  OLI/TIRS  2020   10   14
     18          304  OLI/TIRS  2020   10   30
     19          320  OLI/TIRS  2020   11   15
     20          336  OLI/TIRS  2020   12    1


Tabla generada y adaptada para visualización (2020):

|Número  |Día Juliano  |  Sensor   |Año  |Mes  |Día  |
|:-|:-|:-|:-|:-|:-|
|       1|           16|  OLI/TIRS | 2020|    1|   16|
|       2|           32|  OLI/TIRS | 2020|    2|    1|
|       3|           48|  OLI/TIRS | 2020|    2|   17|
|       4|           64|  OLI/TIRS | 2020|    3|    4|
|       5|           80|  OLI/TIRS | 2020|    3|   20|
|       6|           96|  OLI/TIRS | 2020|    4|    5|
|       7|          112|  OLI/TIRS | 2020|    4|   21|
|       8|          128|  OLI/TIRS | 2020|    5|    7|
|       9|          144|  OLI/TIRS | 2020|    5|   23|
|      10|          160|  OLI/TIRS | 2020|    6|    8|
|      11|          176|  OLI/TIRS | 2020|    6|   24|
|      12|          208|  OLI/TIRS | 2020|    7|   26|
|      13|          224|  OLI/TIRS | 2020|    8|   11|
|      14|          240|  OLI/TIRS | 2020|    8|   27|
|      15|          256|  OLI/TIRS | 2020|    9|   12|
|      16|          272|  OLI/TIRS | 2020|    9|   28|
|      17|          288|  OLI/TIRS | 2020|   10|   14|
|      18|          304|  OLI/TIRS | 2020|   10|   30|
|      19|          320|  OLI/TIRS | 2020|   11|   15|
|      20|          336|  OLI/TIRS | 2020|   12|    1|
|      21|            2|  OLI/TIRS | 2021|    1|    2|

## 4. Balance de energía

- Procesamiento de imágenes satelitales de acuerdo a la metodología establecida en el modelo METRIC.

### 4.1. Descripción

Se ha seguido la metodología empleada en las siguientes investigaciones relacionadas al modelo METRIC:
- **PRINCIPAL** [2007: Satellite-Based Energy Balance for Mapping Evapotranspiration with Internalized Calibration (METRIC) — Model](http://cursosihlla.bdh.org.ar/ET_UNCu_Cursos/Curso_2019/1_Lectura_recomendada/2007_Allen_METRIC_Satellite-Based_Energy_Balance_for_Mapping_Evapotr.pdf) 
- [2019: Estimación de la evapotranspiración de cultivo de maíz bajo riego mediante percepción remota](http://repositorio.imta.mx/handle/20.500.12013/2065)
- [2018: Estimación de la evapotranspiración en los cultivos alrededor del observatorio de Huancayo mediante sensoramiento remoto](https://repositorio.igp.gob.pe/handle/20.500.12816/4631)

<center>
<h3>Base teórica y computacional: Ecuación del balance de energía </h3>
<img src= 'https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Fwww.researchgate.net%2Fprofile%2FBiswajeet_Pradhan%2Fpublication%2F301679158%2Ffigure%2Ffig1%2FAS%3A412214499528704%401475291003507%2FSurface-Energy-Balance-12.jpg&f=1&nofb=1' width=50%/>
</center>

$$
LE = R_n - G - H
$$

Dónde: 
- $LE$ : Energía latente consumida por ET $[W/m²]$
- $R_n$ : Radiación neta (suma de todas las radiaciones de onda corta y larga, sean entrantes o salientes a la superficie) $[W/m²]$
- $G$ : Flujo de calor sensible conducido al suelo $[W/m²]$
- $H$ : Flujo de calor sensible convectado al aire $[W/m²]$



De esta ecuación se basa ***SEBAL***. 

***METRIC*** en comparación realiza una calibración interna enfocada en el parámetro *H* con el fil de absorber la mayor cantidad de errores de estimación y sesgos.  

En METRIC:
- $R_n$ is computed from the satellite-measured narrow-band reflectance and surface temperature
- $G$ is estimated from $R_n$, surface temperature, and vegetation indices
- and $H$ is estimated from surface temperature ranges, surface roughness,
and wind speed using buoyancy corrections.

### 4.2. Radiación Neta / Net Radiation $R_n$

Morse et al. 2000:

$$
R_n = (1- \alpha)R_{S\downarrow} + (R_{L\downarrow} - R_{L\uparrow}) - (1- \epsilon_0)R_{L\downarrow}
$$

Dónde: 
- $R_n$ : Flujo de radiación neta $[W/m^2]$
- $\alpha$ : Albedo de superficie
- $R_{S\downarrow}$ : Radiación de onda corta entrante $[W/m^2]$
- $R_{L\downarrow}$ : Radiación de onda larga entrante $[W/m^2]$
- $R_{L\uparrow}$ : Radiación de onda larga saliente $[W/m^2]$
- $\epsilon_0$ : Emisividad del ancho de banda en la superficie / broad-band surface thermal emissivity

The $(1- \epsilon_0)R_{L\downarrow}$ term represents the fraction of incoming long-wave
radiation reflected from the surface.

#### Albedo / Surface Albedo $\alpha_{s}$

El albedo es la **proporción de radiación incidente que es reflejada por la superficie terrestre**, un método para obtener el valor del albedo se describe en el trabajo de Morse, Allen, y Kramber (2000), mediante la siguiente ecuación la cual está en función de la elevación, y diversas condiciones expresadas en las siguientes ecuaciones.

$$
\alpha = \frac{\alpha_{toa}-\alpha_{pr}}{\tau_{sw}^2} 
$$

Dónde:
- $\alpha$ : Albedo de la superficie / Surface albedo
- $\alpha_{toa}$ : Albedo en el límite superior de la atmósfera / Albedo at the top of the atmosphere (planetary)
- $\alpha_{pr}$ : Albedo de trayectoria, igual a 0.03 (Bastiaanssen, 2000) / Albedo atmosférico ✔️
- $\tau_{sw}$ : Transmitancia en un sentido con condiciones de claridad $[W/m^2K]$ / Transmisividad atmosférica
- $z$ : Elevación sobre el nivel del mar $[m]$, del modelo digital de elevaciones

Ejemplo de estimación del albedo:
- [2018: Procedures for calculation of the albedo with OLI-Landsat 8 images: Application to the Brazilian semi-arid](https://www.scielo.br/j/rbeaa/a/sX6cJjNXWMfHQ5p4h33B8Zz/?lang=en)

Para estimar $\tau_{sw}$:

$$
\tau_{sw} = \tau_B + \tau_D
$$

Donde:
- $\tau_B$ es el índice de transmisividad de radiación directa y 
- $\tau_D$ es índice de transmisividad de radiación difusa

La ecuación utilizada para el cálculo de $\tau_B$ es:

$$
\tau_B = 0.98 * \exp \left[ \frac{-0.00146*P_{air}}{K_t * \cos\theta} - 0.075 * \left( \frac {W}{\cos\theta}\right)^{0.4} \right]
$$

$P_{air}$, $W$ , $K_t$ y $\cos{θ}$ fueron definidas en las ecuaciones anteriores:

$$
P_{air} = 101.1 \left( \frac {293 - 0.0067 * z}{293}\right)^{5.26}
$$

Donde: $z$ es la elevacion media de la imagen, respecto al nivel del mar. El agua precipitable se calculo mediante la ecuación:

$$
W = 0,14 ∗ e_{a} ∗ P_{air} + 2,1
$$

El índice de transmisividad de radiación difusa es estimó mediante $T_B$ para diferentes valores:

$$
T_D = 0.35-0.36T_B \text{  para  } T_B \geq 0.15 
$$
$\space$
$$
T_D = 0.18-0.82T_B \text{  para  } T_B \lt 0.15 
$$



In [ ]:
# El método numérico de Python exp() devuelve un resultado exponencial de x: e^x
# Para exp usar: math.exp()
a = 3
import math
math.exp(a)

- Coeficientes de ponderación de la irradiancia exoatmosférica de la banda $\lambda$

$$
\alpha_s = \sum_{b=1}^{n}[\rho_{s,b} \times \omega_b]
$$

- $\rho_\lambda$ : Valor de la reflectancia de la banda $\lambda$ ✔️
- $\omega_\lambda$ : Coef. de ponderación de la irradiancia exoatmosférica de la banda $\lambda$ ✔️

In [ ]:
# @markdown - Paleta NDVI (Yuri SIG) 20 Colores: `yuri_ndvi_pal`
yuri_ndvi_pal = [
    'ffffff', 'fffcff', 'fff9ff', 'fff7ff', 'fff4ff', 'fff2ff', 'ffefff',
    'ffecff', 'ffeaff', 'ffe7ff', 'ffe5ff', 'ffe2ff', 'ffe0ff', 'ffddff',
    'ffdaff', 'ffd8ff', 'ffd5ff', 'ffd3ff', 'ffd0ff', 'ffceff', 'ffcbff',
    'ffc8ff', 'ffc6ff', 'ffc3ff', 'ffc1ff', 'ffbeff', 'ffbcff', 'ffb9ff',
    'ffb6ff', 'ffb4ff', 'ffb1ff', 'ffafff', 'ffacff', 'ffaaff', 'ffa7ff',
    'ffa4ff', 'ffa2ff', 'ff9fff', 'ff9dff', 'ff9aff', 'ff97ff', 'ff95ff',
    'ff92ff', 'ff90ff', 'ff8dff', 'ff8bff', 'ff88ff', 'ff85ff', 'ff83ff',
    'ff80ff', 'ff7eff', 'ff7bff', 'ff79ff', 'ff76ff', 'ff73ff', 'ff71ff',
    'ff6eff', 'ff6cff', 'ff69ff', 'ff67ff', 'ff64ff', 'ff61ff', 'ff5fff',
    'ff5cff', 'ff5aff', 'ff57ff', 'ff55ff', 'ff52ff', 'ff4fff', 'ff4dff',
    'ff4aff', 'ff48ff', 'ff45ff', 'ff42ff', 'ff40ff', 'ff3dff', 'ff3bff',
    'ff38ff', 'ff36ff', 'ff33ff', 'ff30ff', 'ff2eff', 'ff2bff', 'ff29ff',
    'ff26ff', 'ff24ff', 'ff21ff', 'ff1eff', 'ff1cff', 'ff19ff', 'ff17ff',
    'ff14ff', 'ff12ff', 'ff0fff', 'ff0cff', 'ff0aff', 'ff07ff', 'ff05ff',
    'ff02ff', 'ff00ff', 'ff00ff', 'ff0af4', 'ff15e9', 'ff1fdf', 'ff2ad4',
    'ff35c9', 'ff3fbf', 'ff4ab4', 'ff55aa', 'ff5f9f', 'ff6a94', 'ff748a',
    'ff7f7f', 'ff8a74', 'ff946a', 'ff9f5f', 'ffaa55', 'ffb44a', 'ffbf3f',
    'ffc935', 'ffd42a', 'ffdf1f', 'ffe915', 'fff40a', 'ffff00', 'ffff00',
    'fffb00', 'fff700', 'fff300', 'fff000', 'ffec00', 'ffe800', 'ffe400',
    'ffe100', 'ffdd00', 'ffd900', 'ffd500', 'ffd200', 'ffce00', 'ffca00',
    'ffc600', 'ffc300', 'ffbf00', 'ffbb00', 'ffb700', 'ffb400', 'ffb000',
    'ffac00', 'ffa800', 'ffa500', 'ffa500', 'f7a400', 'f0a300', 'e8a200',
    'e1a200', 'd9a100', 'd2a000', 'ca9f00', 'c39f00', 'bb9e00', 'b49d00',
    'ac9c00', 'a59c00', '9d9b00', '969a00', '8e9900', '879900', '7f9800',
    '789700', '709700', '699600', '619500', '5a9400', '529400', '4b9300',
    '439200', '349100', '2d9000', '258f00', '1e8e00', '168e00', '0f8d00',
    '078c00', '008c00', '008c00', '008700', '008300', '007f00', '007a00',
    '007600', '007200', '006e00', '006900', '006500', '006100', '005c00',
    '005800', '005400', '005000', '004c00'
]

In [ ]:
image_ID = ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_010065_20200116')

In [ ]:
# Albedo 
image = image_ID.multiply(0.0001)

# Coeficientes de ponderación por banda (Da Silva et al., 2016)
# https://www.scielo.br/j/rbeaa/a/sX6cJjNXWMfHQ5p4h33B8Zz/?lang=en

albedo = image.expression(
    '(0.3*B2 + 0.277*B3 + 0.233*B4 + 0.143*B5 + 0.036*B6 + 0.012*B7)',
    {
        'B2' : image.select('B2'),
        'B3' : image.select('B3'),
        'B4' : image.select('B4'),
        'B5' : image.select('B5'),
        'B6' : image.select('B6'),
        'B7' : image.select('B7')
    }
)

palette = [
    'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
    '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
    '012E01', '011D01', '011301'
  ]
viz_params = {'min': 0.0, 'max': 0.5, 'palette': palette};

mapdisplay([-79.309, -6.846], {'Imagen':image.getMapId({'min':0, 'max':0.3, 'bands':['B4','B3','B2']}),
                               'Albedo':albedo.getMapId(viz_params)}, zoom_start=9)

# mapdisplay([-79.309, -6.846], {'Albedo':albedo.clip(roiDep).getMapId(viz_params)},zoom_start=8)

#### Radiación de onda corta entrante / Incoming Solar Radiation $R_{S\downarrow}$ 

$R_{S\downarrow}$ : Radiación de onda corte entrante $[W/m²]$

Morse et al. (2000)

$$
R_{S\downarrow} = \frac{G_{sc} \cos\theta_{rel} \tau_{sw}}{d^2}
$$

Dónde:
-
- $G_{sc}$ : Constante solar $[1367 \space W/m²]$ ✔️
- $\theta$ : Ángulo zenital solar / Ángulo de incidencia solar $[rad]$ ✔️ 
- $d^2$ : Cuadrado de la distancia relativa Tierra-Sol $[m^{-1}]$ ✔️ (OJO: en 2018 se menciona el inverso del cuadrado de la distancia relativa de tierra al sol)
- $\tau_{sw}$ : Es la transmitancia en un sentido con condiciones de claridad / Transmitancia de la banda ancha $[W/m² \space K]$

In [ ]:
image_ID.getInfo()

In [ ]:
import math

ang_zenital = image_ID.get('SOLAR_ZENITH_ANGLE').getInfo()*math.pi/180
G_sc = 1367
dr = 1/image_ID.get('EARTH_SUN_DISTANCE').getInfo()

In [ ]:
1/0.983642

In [ ]:
# imagen = ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_010065_20200116')
zenitAngle = image.get('SOLAR_ZENITH_ANGLE').getInfo()
zenitAngle

In [ ]:
R_s_entrante = 1367*cos(zenitAngle)*d_r*tau_sw

#### Radiación de onda larga saliente / Outgoing Long-Wave Radiation $R_{L\uparrow}$ 

Outgoing long-wave radiation, RL↑, emitted from the surface is
driven by surface temperature and surface emissivity. RL↑ is computed using the Stefan–Boltzmann equation

Morse et al., 2000.

$$
R_{L\downarrow} = \epsilon_0 * \sigma * T_{s}^{4}
$$

Dónde:
- $R_{L\downarrow}$ : Radiación de onda larga saliente $[W/m²]$
- $\epsilon_0$ : Emisividad del ancho de banda de la superficie
- $\sigma$ : Constante de Stefan-Boltzmann $(5.67*10^{-8}[W m^{-2}K^{-4}])$
- $T_s$ : Temperatura de brillo de superficie $[K]$

Para hallar la emisividad:

$$
\epsilon_0 = 0.95 + 0.01 * LAI \text{, cuando NDVI > 0 y LAI < 3} 
$$

Dónde:
- $\text{LAI}$ : es el índice de área foliar
- $\text{NDVI}$ : El índice de vegetación de diferencia normalizada

In [ ]:
R_l_saliente = emisividad*(5.67*10**-8)*temp_brillo**4

#### Radiación de onda larga entrante / Incoming Long-Wave Radiation $R_{L\downarrow}$

Incoming long-wave radiation is the downward thermal radiation
flux originating from the atmosphere W m−2 and is traditionally
computed using the Stefan–Boltzmann equation

Bastiaansen et al. 1998:

$$
R_{L\downarrow} = \epsilon_a \sigma T_{0 ref}^{4}
$$

Dónde:
- $\epsilon_a$ : Emisividad atmosférica efectiva
- $\sigma$ : Constante de Stefan-Boltzmann $(5.67*10^{-8} [W/m^{2}])$
- $T_{0 ref}$ : Temperatura en un punto de referencia con buen riego / near-surface air temperature $[K]$

> Ecuación empírica para $\epsilon_a$ de Bastiaanssen (1995) aplicado con coeficientes desarrollados por Allen et al. (2000) con datos recolectados de **alfalfa** en Idaho:
$$
\epsilon_a = 0.85(-\ln{\tau_{sw}})^{0.09}
$$

> The original coefficients
by Bastiaanssen (1995), derived for western Egypt, were
$$
\epsilon_a = 1.08 * (-\ln{\tau_{sw}})^{0.265}
$$

Dónde:
- $R_{L\downarrow}$ : Radiación de onda corta entrante
- $\tau_{sw}$ : Es la transmitancia en un sentido con condiciones de claridad



#### Radiación neta de la superficie $R_n$

### 4.3. Flujo de calor del suelo / Soil Heat Flux $G$

> **Forma 1**
>
>Choudhury, Idso y Reginato (1987) o Allen, Pereira, Raes y Smith (1998)
>
>$$
G = 0.4 * \mathrm{e}^{-0.5* \text{LAI}} * R_n
$$
>
>Dónde:
>- $G$ : Flujo de calor del suelo $[W/m^{2}]$
>- $\text{LAI}$ : Índice de área foliar
>- $R_n$ : Flujo de radiación neta $[W/m^{2}]$

> **Forma 2**
>
> Despeje de la relación de calor almacenado en el suelo y vegetación debida a la conducción $R_n$, según Bastiaanssen (2000)
>
>$$
\frac{G}{R_n} = \frac{T_s}{\alpha}(0.0038\alpha + 0.0074\alpha^{2})(1-0.98*\text{NDVI}^4) 
$$
> Dónde:
> - $\frac{G}{R_n}$ : Relación del calor almacenado en suelo y vegetación
> - $T_s$ : Temperatura de brillo de superficie $[K]$
> - $\alpha$ : Albedo de superficie
> - $\text{NDVI}$ : Índice de vegetación de diferencia normalizada

#### Momento de resistencia aerodinámica (MRA) o $Z_{om}$

Chemin y Din Ahmad (2000), toman en cuenta las condiciones aerodinámicas de la vegetación la cual está en función de su altura y distribución espacial.

$$
Z_{om} = a + b*\text{NDVI}
$$

Dónde:

$$
a = \ln(0.02) - [b*(0.02)]
$$

$$
b = \frac{(\text{NDVI}_{max}-0.02)}{\ln(\frac{h_v}{7}-\ln(0.002)}
$$

- $\text{NDVI}$ : Índice de vegetación de diferencia normalizada
- $\text{NDVI}_{max}$ : Valor local máximo del $\text{NDVI}$
- $h_v$ : altura promedio de la vegetación cuando se tiene $\text{NDVI}_{max}$

#### Selección de pixel caliente y frío

Requisito para conocer el aporte de flujo de calor sensible $H$ en el modelo de balance, cada pixel con vegetación requerirá de conocer un gradiente de temperatura $\delta T$ en referencia a su condición aerodinámica a fin de estimar la densidad de su aporte, esto se logra a partir de la relación lineal que guardan las temperaturas entre dos superficies extremas con características distintas. 

Se han propuesto metodologías para la selección de estos puntos extremos denominándose **pixel frío** y **pixel caliente**, donde: 
- un **pixel frío** representa una superficie con vegetación densa y húmeda, mientras 
- un **pixel caliente** representa una superficie seca sin vegetación. 

Según el método de Olmedo et al. (2016), la selección de estos dos parámetros puede llevarse sobre el área de estudio y localizarlos mediante el cumplimiento de una serie de condiciones sobre otros parámetros conocidos como se muestra a continuación:

$\space$

$$
\text{pixel frio} = 
\begin{cases}
  \begin{gather*}
    3 \le \text{LAI} \le 6 \\
    0.18 \le \alpha \le 0.25 \\
    0.03 \le Z_{0m} \le 0.08 \\
    \text{NDVI} \ge \text{NDVI}_{max} - 0.15 \\
    T_s < T_{max} - 5 
  \end{gather*}
\end{cases}
$$

$\space$

$$
\text{pixel caliente} = 
\begin{cases}
  \begin{gather*}
    0.13 \le \alpha \le 0.15 \\
    Z_{0m} \le 0.005 \\
    0.1 \le \text{NDVI} \le 0.28 \\
    T_s > T_{max} - 5
  \end{gather*}
\end{cases}
$$

Dónde:
- $\alpha$ : Es el albedo de superficie
- $\text{LAI}$ : Índice de área foliar
- $Z_{0m}$ : Momento de resistencia aerodinámica o MRA
- $\text{NDVI}$ : Índice de vegetación de diferencia normalizada y máximo $\text{NDVI}_{max}$
- $T_s$ : Temperatura de superficie y máximo valor $T_{max}$


### 4.4. Flujo de calor sensible / Sensible Heat Flux $H$

Bastiaanssen et al. (1998)

$$
H = \frac{\rho_a c_p \delta{T_a}}{r_{ah}}
$$

Dónde:
- $H$ : Flujo de calor sensible $[W/m^{2}]$
- $\rho_a$ : Densidad del aire $[kg/m^{3}]$
- $c_p$ : Calor específico del aire igual a $[1004 \space J/kg/K]$
- $\delta{T_a}$ : Gradiente de temperatura $[°C]$
- $r_{ah}$ : Resistencia aerodinámica al transporte de calor $[s/m]$ 

### 4.5. $\lambda{ET}$ y $ET$ instantánea

### 4.6. Evapotranspiración de cultivo $ET_c$ por día

La $ET_c$ diaria se puede calcular mediante la siguiente ecuación:

$$
ET_c[\frac{mm}{d}] = C_{rad} * \Lambda * ET_{24}
$$

## 5. Pruebas

- Pruebas realizadas durante la investigación.

#### Diccionarios generados

Para darle buen formato a los diccionarios, usar:
- [Python Formatter](https://codebeautify.org/python-formatter-beautifier)

In [ ]:
# @markdown Diccionario Sentinel 2: `imgCol_dict`
imgCol_dict = {
    1: [
        "COPERNICUS/S2_SR/20200102T153611_20200102T153743_T17MPN",
        "COPERNICUS/S2_SR/20200107T153619_20200107T154047_T17MPN",
        "COPERNICUS/S2_SR/20200112T153611_20200112T153613_T17MPN",
        "COPERNICUS/S2_SR/20200117T153619_20200117T153613_T17MPN",
        "COPERNICUS/S2_SR/20200122T153611_20200122T153612_T17MPN",
        "COPERNICUS/S2_SR/20200127T153609_20200127T153611_T17MPN",
    ],
    2: [
        "COPERNICUS/S2_SR/20200201T153611_20200201T153611_T17MPN",
        "COPERNICUS/S2_SR/20200206T153619_20200206T153613_T17MPN",
        "COPERNICUS/S2_SR/20200211T153611_20200211T153626_T17MPN",
        "COPERNICUS/S2_SR/20200216T153619_20200216T154305_T17MPN",
        "COPERNICUS/S2_SR/20200221T153611_20200221T154337_T17MPN",
        "COPERNICUS/S2_SR/20200226T153619_20200226T154119_T17MPN",
    ],
    3: [
        "COPERNICUS/S2_SR/20200302T153611_20200302T153614_T17MPN",
        "COPERNICUS/S2_SR/20200307T153619_20200307T153616_T17MPN",
        "COPERNICUS/S2_SR/20200312T153621_20200312T153615_T17MPN",
        "COPERNICUS/S2_SR/20200317T153619_20200317T154424_T17MPN",
        "COPERNICUS/S2_SR/20200322T153621_20200322T154411_T17MPN",
        "COPERNICUS/S2_SR/20200327T153619_20200327T154304_T17MPN",
    ],
    4: [
        "COPERNICUS/S2_SR/20200401T153621_20200401T153901_T17MPN",
        "COPERNICUS/S2_SR/20200406T153619_20200406T154234_T17MPN",
        "COPERNICUS/S2_SR/20200411T153621_20200411T153910_T17MPN",
        "COPERNICUS/S2_SR/20200416T153619_20200416T154403_T17MPN",
        "COPERNICUS/S2_SR/20200421T153621_20200421T154337_T17MPN",
        "COPERNICUS/S2_SR/20200426T153619_20200426T153904_T17MPN",
    ],
    5: [
        "COPERNICUS/S2_SR/20200501T153621_20200501T153850_T17MPN",
        "COPERNICUS/S2_SR/20200506T153619_20200506T153616_T17MPN",
        "COPERNICUS/S2_SR/20200511T153621_20200511T154343_T17MPN",
        "COPERNICUS/S2_SR/20200516T153619_20200516T153618_T17MPN",
        "COPERNICUS/S2_SR/20200516T153619_20200516T154501_T17MPN",
        "COPERNICUS/S2_SR/20200521T153631_20200521T154352_T17MPN",
        "COPERNICUS/S2_SR/20200526T153619_20200526T154127_T17MPN",
        "COPERNICUS/S2_SR/20200531T153631_20200531T153940_T17MPN",
    ],
    6: [
        "COPERNICUS/S2_SR/20200605T153619_20200605T153834_T17MPN",
        "COPERNICUS/S2_SR/20200610T153631_20200610T154258_T17MPN",
        "COPERNICUS/S2_SR/20200615T153619_20200615T153722_T17MPN",
        "COPERNICUS/S2_SR/20200620T153631_20200620T154446_T17MPN",
        "COPERNICUS/S2_SR/20200625T153619_20200625T153621_T17MPN",
        "COPERNICUS/S2_SR/20200625T153619_20200625T154504_T17MPN",
        "COPERNICUS/S2_SR/20200630T153621_20200630T153946_T17MPN",
    ],
    7: [
        "COPERNICUS/S2_SR/20200705T153619_20200705T153621_T17MPN",
        "COPERNICUS/S2_SR/20200710T153621_20200710T154423_T17MPN",
        "COPERNICUS/S2_SR/20200715T153619_20200715T153620_T17MPN",
        "COPERNICUS/S2_SR/20200715T153619_20200715T154503_T17MPN",
        "COPERNICUS/S2_SR/20200720T153621_20200720T154406_T17MPN",
        "COPERNICUS/S2_SR/20200725T153619_20200725T153621_T17MPN",
        "COPERNICUS/S2_SR/20200730T153631_20200730T154345_T17MPN",
    ],
    8: [
        "COPERNICUS/S2_SR/20200804T153619_20200804T154255_T17MPN",
        "COPERNICUS/S2_SR/20200809T153631_20200809T154345_T17MPN",
        "COPERNICUS/S2_SR/20200814T153619_20200814T153738_T17MPN",
        "COPERNICUS/S2_SR/20200819T153631_20200819T153625_T17MPN",
        "COPERNICUS/S2_SR/20200824T153619_20200824T154230_T17MPN",
        "COPERNICUS/S2_SR/20200829T153621_20200829T154210_T17MPN",
    ],
    9: [
        "COPERNICUS/S2_SR/20200903T153619_20200903T154357_T17MPN",
        "COPERNICUS/S2_SR/20200908T153621_20200908T154046_T17MPN",
        "COPERNICUS/S2_SR/20200913T153619_20200913T153628_T17MPN",
        "COPERNICUS/S2_SR/20200918T153621_20200918T153624_T17MPN",
        "COPERNICUS/S2_SR/20200923T153619_20200923T153831_T17MPN",
        "COPERNICUS/S2_SR/20200928T153621_20200928T153624_T17MPN",
    ],
    10: [
        "COPERNICUS/S2_SR/20201003T153619_20201003T153759_T17MPN",
        "COPERNICUS/S2_SR/20201008T153631_20201008T153625_T17MPN",
        "COPERNICUS/S2_SR/20201013T153619_20201013T153807_T17MPN",
        "COPERNICUS/S2_SR/20201018T153621_20201018T153841_T17MPN",
        "COPERNICUS/S2_SR/20201023T153619_20201023T154139_T17MPN",
        "COPERNICUS/S2_SR/20201028T153621_20201028T154210_T17MPN",
    ],
    11: [
        "COPERNICUS/S2_SR/20201102T153619_20201102T154334_T17MPN",
        "COPERNICUS/S2_SR/20201107T153621_20201107T154039_T17MPN",
        "COPERNICUS/S2_SR/20201112T153619_20201112T154104_T17MPN",
        "COPERNICUS/S2_SR/20201117T153621_20201117T154056_T17MPN",
        "COPERNICUS/S2_SR/20201122T153619_20201122T154415_T17MPN",
        "COPERNICUS/S2_SR/20201127T153621_20201127T154054_T17MPN",
    ],
    12: [
        "COPERNICUS/S2_SR/20201202T153619_20201202T154414_T17MPN",
        "COPERNICUS/S2_SR/20201207T153621_20201207T154054_T17MPN",
        "COPERNICUS/S2_SR/20201212T153619_20201212T154111_T17MPN",
        "COPERNICUS/S2_SR/20201222T153619_20201222T154433_T17MPN",
        "COPERNICUS/S2_SR/20201227T153621_20201227T154110_T17MPN",
    ],
}

In [ ]:
# @markdown Diccionario Landsat 8 TOA: `imgCol_dict`

imgCol_dict = {
    1: ['LANDSAT/LC08/C01/T1_TOA/LC08_010065_20200116'],
    2: ['LANDSAT/LC08/C01/T1_TOA/LC08_010065_20200201',
      'LANDSAT/LC08/C01/T1_TOA/LC08_010065_20200217'],
    3: ['LANDSAT/LC08/C01/T1_TOA/LC08_010065_20200304',
      'LANDSAT/LC08/C01/T1_TOA/LC08_010065_20200320'],
    4: ['LANDSAT/LC08/C01/T1_TOA/LC08_010065_20200405',
      'LANDSAT/LC08/C01/T1_TOA/LC08_010065_20200421'],
    5: ['LANDSAT/LC08/C01/T1_TOA/LC08_010065_20200507',
      'LANDSAT/LC08/C01/T1_TOA/LC08_010065_20200523'],
    6: ['LANDSAT/LC08/C01/T1_TOA/LC08_010065_20200608',
      'LANDSAT/LC08/C01/T1_TOA/LC08_010065_20200624'],
    7: ['LANDSAT/LC08/C01/T1_TOA/LC08_010065_20200726'],
    8: ['LANDSAT/LC08/C01/T1_TOA/LC08_010065_20200811',
      'LANDSAT/LC08/C01/T1_TOA/LC08_010065_20200827'],
    9: ['LANDSAT/LC08/C01/T1_TOA/LC08_010065_20200912',
      'LANDSAT/LC08/C01/T1_TOA/LC08_010065_20200928'],
    10: ['LANDSAT/LC08/C01/T1_TOA/LC08_010065_20201014',
      'LANDSAT/LC08/C01/T1_TOA/LC08_010065_20201030'],
    11: ['LANDSAT/LC08/C01/T1_TOA/LC08_010065_20201115'],
    12: ['LANDSAT/LC08/C01/T1_TOA/LC08_010065_20201201']
 }

# @markdown Diccionario Landsat 8 Raw Scenes: `imgCol_dict`

imgCol_dict = {
    1: ['LANDSAT/LC08/C01/T1/LC08_010065_20200116'],
    2: ['LANDSAT/LC08/C01/T1/LC08_010065_20200201',
      'LANDSAT/LC08/C01/T1/LC08_010065_20200217'],
    3: ['LANDSAT/LC08/C01/T1/LC08_010065_20200304',
      'LANDSAT/LC08/C01/T1/LC08_010065_20200320'],
    4: ['LANDSAT/LC08/C01/T1/LC08_010065_20200405',
      'LANDSAT/LC08/C01/T1/LC08_010065_20200421'],
    5: ['LANDSAT/LC08/C01/T1/LC08_010065_20200507',
      'LANDSAT/LC08/C01/T1/LC08_010065_20200523'],
    6: ['LANDSAT/LC08/C01/T1/LC08_010065_20200608',
      'LANDSAT/LC08/C01/T1/LC08_010065_20200624'],
    7: ['LANDSAT/LC08/C01/T1/LC08_010065_20200726'],
    8: ['LANDSAT/LC08/C01/T1/LC08_010065_20200811',
      'LANDSAT/LC08/C01/T1/LC08_010065_20200827'],
    9: ['LANDSAT/LC08/C01/T1/LC08_010065_20200912',
      'LANDSAT/LC08/C01/T1/LC08_010065_20200928'],
    10: ['LANDSAT/LC08/C01/T1/LC08_010065_20201014',
      'LANDSAT/LC08/C01/T1/LC08_010065_20201030'],
    11: ['LANDSAT/LC08/C01/T1/LC08_010065_20201115'],
    12: ['LANDSAT/LC08/C01/T1/LC08_010065_20201201']
}

In [ ]:
# @markdown Diccionario Landsat 7: `imgCol_dict_L7`

imgCol_dict = {
    1: ['LANDSAT/LE07/C01/T1_SR/LE07_010065_20200108',
        'LANDSAT/LE07/C01/T1_SR/LE07_010065_20200124'],
    2: ['LANDSAT/LE07/C01/T1_SR/LE07_010065_20200209',
      'LANDSAT/LE07/C01/T1_SR/LE07_010065_20200225'],
    3: ['LANDSAT/LE07/C01/T1_SR/LE07_010065_20200312',
      'LANDSAT/LE07/C01/T1_SR/LE07_010065_20200328'],
    4: ['LANDSAT/LE07/C01/T1_SR/LE07_010065_20200429'],
    5: ['LANDSAT/LE07/C01/T1_SR/LE07_010065_20200515',
      'LANDSAT/LE07/C01/T1_SR/LE07_010065_20200531'],
    6: [],
    7: ['LANDSAT/LE07/C01/T1_SR/LE07_010065_20200702',
      'LANDSAT/LE07/C01/T1_SR/LE07_010065_20200718'],
    8: ['LANDSAT/LE07/C01/T1_SR/LE07_010065_20200803',
      'LANDSAT/LE07/C01/T1_SR/LE07_010065_20200819'],
    9: ['LANDSAT/LE07/C01/T1_SR/LE07_010065_20200904',
      'LANDSAT/LE07/C01/T1_SR/LE07_010065_20200920'],
    10: ['LANDSAT/LE07/C01/T1_SR/LE07_010065_20201006',
      'LANDSAT/LE07/C01/T1_SR/LE07_010065_20201022'],
    11: ['LANDSAT/LE07/C01/T1_SR/LE07_010065_20201107',
      'LANDSAT/LE07/C01/T1_SR/LE07_010065_20201123'],
    12: ['LANDSAT/LE07/C01/T1_SR/LE07_010065_20201209',
      'LANDSAT/LE07/C01/T1_SR/LE07_010065_20201225']
  }

#### Obtener fechas 

Otra forma de obtener fechas

- Desventaja: Requiere `pprint` para una mejor visualización del resultado (lista).

In [ ]:
# Cargar una imagen Landsat 5 TM
collection = ee.ImageCollection("LANDSAT/LT05/C01/T1")\
               .filterDate("2009-01-01","2010-01-01")\
               .filterMetadata('WRS_PATH', 'equals', 3)\
               .filterMetadata('WRS_ROW', 'equals', 68)

In [ ]:
# Funcion de extraer fecha
from pprint import pprint
def list_coll(coll):
  '''
  https://developers.google.com/earth-engine/apidocs/ee-date-format?hl=en
  '''
  func_ejv = lambda img: ee.String(ee.Image(img).date().format('YYYY-MM-dd H:mm:ss')).slice(0)
  return pprint(coll.toList(coll.size(), 0).map(func_ejv).getInfo())

# Problema: Retorna las fechas 

In [ ]:
list_coll(collection)

# Bibliografía

- Links de interés

- Allen, R. G. (2007). Satellite-Based Energy Balance for Mapping Evapotranspiration with Internalized Calibration (METRIC) - Model. Journal of Irrigation and Drainage Engineering, 133 (Noviembre 2015), 380-394. DOI: 10.1061/(ASCE)0733-9437(2007)133.
- Kilic, A., Allen, R., Huntington, J., y Ozturk, D. (2018). Earth Engine Evapotranspiration Flux v.0.10.4. Recuperado el 1 de julio de 2018, a partir de http://eeflux-level1.appspot.com/

**Búsquedas de METRIC Evapotranspiration**

- [Búsqueda en *ScienceDirect*: METRIC Evapotranspiration](https://www.sciencedirect.com/search?qs=metric%20evapotranspiration)
- [Búsqueda en *MDPI*: METRIC Evapotranspiration](https://www.mdpi.com/search?q=metric+evapotranspiration)

- [NASA Evapotranspiration Data Products and Applications](http://www.cazalac.org/mwar_lac/fileadmin/imagenes2/Remote_Sensing/S4P1_light.pdf)

Seguimiento de Metodología:

> [ConectedPapers - 2007: Satellite-Based Energy Balance for Mapping Evapotranspiration with Internalized Calibration (METRIC) — Model](https://www.connectedpapers.com/main/817edad756d41499da26498e71c85afaa884383a/SatelliteBased-Energy-Balance-for-Mapping-Evapotranspiration-with-Internalized-Calibration-METRICModel/graph)

- **PRINCIPAL** [2007: Satellite-Based Energy Balance for Mapping Evapotranspiration with Internalized Calibration (METRIC) — Model](http://cursosihlla.bdh.org.ar/ET_UNCu_Cursos/Curso_2019/1_Lectura_recomendada/2007_Allen_METRIC_Satellite-Based_Energy_Balance_for_Mapping_Evapotr.pdf) 
- [2019: Estimación de la evapotranspiración de cultivo de maíz bajo riego mediante percepción remota](http://repositorio.imta.mx/handle/20.500.12013/2065)
- [2018: Estimación de la evapotranspiración en los cultivos alrededor del observatorio de Huancayo mediante sensoramiento remoto](https://repositorio.igp.gob.pe/handle/20.500.12816/4631)

<img src='https://mms.businesswire.com/media/20180706005099/en/666704/4/INTECHOPEN_LOGO_RED_RGB.jpg?download=1' width=4%>IntechOpen:
- [2011: Operational Remote Sensing of ET and Challenges](https://www.intechopen.com/books/evapotranspiration-remote-sensing-and-modeling/operational-remote-sensing-of-et-and-challenges)
- [2018: Influence of Landsat Revisit Frequency on Time-Integration of Evapotranspiration for Agricultural Water Management](https://www.intechopen.com/books/advanced-evapotranspiration-methods-and-applications/influence-of-landsat-revisit-frequency-on-time-integration-of-evapotranspiration-for-agricultural-wa)




Investigadores de ETP:

- [Richard Allen: University of Idaho](https://www.uidaho.edu/cals/soil-and-water-systems/our-people/richard-allen)
- [Ayse Kilic: University of Nebraska - Lincoln](https://engineering.unl.edu/civil/ayse-kilic/)


- [2017: Calibration of METRIC Model to Estimate Energy Balance over a Drip-Irrigated Apple Orchard](https://www.mdpi.com/2072-4292/9/7/670/htm)
- [2019: LandMOD ET mapper: A new matlab-based graphical user interface (GUI) for automated implementation of SEBAL and METRIC models in thermal imagery](https://www.sciencedirect.com/science/article/abs/pii/S1364815219301203)

**Recursos de $\LaTeX$ empleados**
- [LaTex mathematics for equations](https://en.wikibooks.org/wiki/LaTeX/Mathematics)
- [Overleaf - Aligning equations with amsmath](https://www.overleaf.com/learn/latex/Aligning_equations_with_amsmath)
- [Wumbo - Mathematical Symbols](https://wumbo.net/symbols/)



**Tutoriales**
- [Conversión de valores de ND a radiancia](https://www.youtube.com/watch?v=kRN1ekslBH0&list=PLkMtpdwUewHG5JvNoZbOL7-c6_EXoS_UI)

In [ ]:
# @markdown Typical Landsat Collection 2 Product Generation
%%html
<p align='center'>
  <img src='https://prd-wret.s3.us-west-2.amazonaws.com/assets/palladium/production/s3fs-public/thumbnails/image/Landsat%20Collection%202%20timeline-web.jpg' width='90%'>
</p>